# Sentiment Analysis
## The FAST AI way

In [ ]:
from fastai.text import *
import html

In [ ]:
import fastai.text as f
import torch as t

In [ ]:
from fastai.text import LanguageModelPreLoader

In [ ]:
dir(LanguageModelPreLoader.__doc__)

In [ ]:
PATH = Path("tsa/")

In [ ]:
path_to_file = "twitter-airline-sentiment/"
file_name = "Tweets.csv"

LM_PATH = 'tsa/data/tweeter_lm'

CLS_PATH = 'tsa/data/tweeter_cls'

In [ ]:
# Read csv
raw_data = pd.read_csv(path_to_file + file_name)

In [ ]:
raw_data.head()

In [ ]:
raw_data.info()

In [ ]:
# the goal if the challenge is to find classify a tweet sentiment based on text
# so we can discard the rest of the data
filtered_data  = raw_data[['airline_sentiment', 'text']]
filtered_data.head()

In [ ]:
# convert sentiment text into numeric
def s_t_n(sentiment):
    if sentiment == "negative":
        return -1
    elif sentiment == "positive":
        return 1
    else:
        return 0

filtered_data['airline_sentiment'] = filtered_data['airline_sentiment'].apply(lambda x: s_t_n(x))
filtered_data.head()

In [ ]:
col_names=['labels', 'texts']

In [ ]:
trn_text, val_text = sklearn.model_selection.train_test_split(
    filtered_data, test_size=0.1)

In [ ]:
text = filtered_data['text'].tolist()
df_trn = pd.DataFrame({'texts': text, 'labels':[0]*len(text)})

df_trn.to_csv(LM_PATH +'/train.csv', header='False', index=False)

In [ ]:
df_trn.head()

In [ ]:
# save the filted data
filtered_data.to_csv(CLS_PATH +'/train.csv', header='False', index=False)

## Language Model Tokens

In [ ]:
chunksize=24000

In [ ]:
BOS = 'xbos' # beginning of sentence tag
FLD = 'xfld' # data field tag

In [ ]:
# plagerising from Jeremy my fixup replaces imoticons with something else
re1 = re.compile(r'  +')
def fixup(x):
    x = x.replace("@", " ").replace("http", "urlink")
    return re1.sub(' ', html.unescape(x))

In [ ]:
def get_tokens(df, n_lbs=1):
    labels = df.iloc[:,range(n_lbs)].values.astype(np.int64)
    texts = f'\n{BOS} {FLD} 1 ' + '\n\n'.join(df[n_lbs].astype(str))
#     for i in range(n_lbs+1, len(df.columns)): texts += f' {FLD} {i-nlbs} ' +  df[i].astype(str)
#     texts.apply(fixup).values.astype(str)
    
    tok = Tokenizer().process_all(partition_by_cores(texts, 4))
    return tok, list(labels)

In [ ]:
def get_all(df, n_lbls=1):
    tok, labels = [], []
    for i, r in enumerate(df):
        tok_, labels_ = get_tokens(r, n_lbls)
        tok += tok_
        labels += labels_
    return tok, labels

In [ ]:
trn = pd.read_csv(LM_PATH + "/train.csv", header=None, chunksize=chunksize)
# trn = pd.read_csv(LM_PATH + "/train.csv", header=None)

In [ ]:
tok, labels = get_all(trn)
# tok, labels = get_tokens(trn)

In [ ]:
' '.join(tok[3])

In [ ]:
np.save(path_to_file + 'lang_tokens.npy', tok)

In [ ]:
tok = np.load(path_to_file + "lang_tokens.npy")
len(tok[0])

In [ ]:
freq = Counter(p for o in tok for p in o)
freq.most_common(25)

In [ ]:
max_vocab = 60000
min_freq = 2

In [ ]:
itos = [o for o, c in freq.most_common(max_vocab) if c>min_freq]
itos.insert(0, '_unk_')
itos.insert(1, '_pad_')

In [ ]:
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(stoi)

In [ ]:
trn_lm = np.array([[stoi[o] for o in p] for p in tok])

In [ ]:
' '.join(str(o) for o in trn_lm[0])

In [ ]:
np.save(path_to_file + 'tok_ids.npy', trn_lm)
pickle.dump(itos, open(path_to_file + 'itos.pkl', 'wb'))

In [ ]:
trn_lm = np.load(path_to_file + 'tok_ids.npy')
itos = pickle.load(open(path_to_file + 'itos.pkl', 'rb'))

In [ ]:
# tok_lm = tok_lm[0] + tok_lm[1] + tok_lm[2] + tok_lm[3]

In [ ]:
vs=len(itos)
vs,len(trn_lm)

In [ ]:
len(trn_lm[2])

# Wikitext103 conversion

In [ ]:
em_sz, nh, nl = 400, 1150, 3

In [ ]:
PRE_PATH = PATH/'models'/'wt103'
PRE_LM_PATH = PRE_PATH/'fwd_wt103.h5'

In [ ]:
wgts = torch.load(PRE_LM_PATH, map_location=lambda storage, loc: storage)

In [ ]:
enc_wgts = to_np(wgts['0.encoder.weight'])
row_m = enc_wgts.mean(0)

In [ ]:
itos2 = pickle.load((PRE_PATH/'itos_wt103.pkl').open('rb'))
stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

In [ ]:
new_w = np.zeros((vs, em_sz), dtype=np.float32)
for i,w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r>=0 else row_m

In [ ]:
wgts['0.encoder.weight'] = Tensor(new_w)
wgts['0.encoder.with_droupout.embed.weight'] = Tensor(np.copy(new_w))
wgts['1.decoder.weight'] = Tensor(np.copy(new_w))

# Language Model

In [ ]:
wd=1e-7
bptt=70
bs=52
opt_fn=partial(optim.Adam, betas=(0.8, 0.99))

In [ ]:
t = len(np.concatenate(trn_lm))
t, t/64

In [ ]:
trn_dl = LanguageModelPreLoader(np.concatenate(trn_lm), bs, bptt)
md = LanguageModelData(PATH, 1, vs, trn_dl, bs=bs,bptt=bptt)

In [ ]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [ ]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
                        dropouti=drops[0], droupout=drops[1], wdrop=drops[2],dropoute=drops[3],dropouth=drops[4])

learner.metrics = [accuracy]
leaner.unfreeze()

In [ ]:
learner.get_layer_groups()

In [ ]:
learner.model.load_state_dict(wgts)

In [ ]:
lr=1e-3
lrs = lr

In [ ]:
learner.fit(lrs/2, 1, wds=wd, use_clr=(32), cycle_len=1)

In [ ]:
learner.save('lm_last_fit')

In [ ]:
learner.load('lm_last_fit')

In [ ]:
learner.unfreeze()

In [ ]:
learner.lr_find(srart_lr=lrs/10, end_lr=lrs*10, linear=True)

In [ ]:
learner.sched.plot()

In [ ]:
learner.fit(lrs, 1, wds=wd, use_clr=(20,10), cycle_len=15)

In [ ]:
learner.save('lm1')

In [ ]:
learner.save_encoder('lm1_enc')

In [ ]:
learner.sched.plot()